Testing the implementaion of All positive values using 8 Qubits - 256 data points. The weights are obtained from pre training using a simulator and tested in the Cat Qubits.

The experimental code for developing Amplitude embedded circuits using Variational Quantum State Preparation (VQSP). The VQSP algorithm uses PQC to create training circuits which learns the parameters to create the corresponding amplitude embedded states.

In [1]:
# !pip install pennylane nftopt

In [2]:
# !pip install git+https://github.com/QuantumETS/pennylane-alice-bob.git@main

We specify the number of qubits needed to map the data, for n qubits we can amplitude embedded 2^n data. The depth represents the ansatz repitition depth.

In [1]:
#defining parameters
n_qubits = 8
depth = 4

In [2]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

We use the Logical Cat qubit version as the physical cat qubit version does not support the gate configurations that have been used in the circuit. Repo Link - <a> https://github.com/QuantumETS/pennylane-alice-bob/tree/main </a>

In [4]:
# dev = qml.device("alicebob.qubit", alice_backend="EMU:40Q:PHYSICAL_CATS", wires=n_qubits, average_nb_photons=4,kappa_1=1e2, kappa_2=1e4, shots=1000)
dev = qml.device("alicebob.qubit", alice_backend="EMU:15Q:LOGICAL_EARLY", wires=n_qubits)
dev

Using alice & bob EMU:15Q:LOGICAL_EARLY backend...


<RemoteDevice device (wires=8, shots=1024) at 0x7e9d6ff91e40>

A single Ansatz layer used for training contains set of trainiable RY gates for all the qubits followed by cyclic CZ arrangement.

In [5]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
        for wire in range(n_qubits):
            qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

        for i in range(n_qubits-1):
            qml.CZ(wires=[i,i+1])
        qml.CZ(wires=[n_qubits-1,0])

We use the Fidelity as the accuracy metric in this case. The Fidelity estimate provides the similarity between the generated state and the required amplitude embedding. It returs 1 if there are similar.

In [6]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [7]:
@qml.qnode(dev)
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  # qml.state() applies Ua to Ini State, which gives Appro_a
  # return qml.state()
  return qml.probs()

In [8]:
def variational_classifier(weights, x):
    # weights are thetas
    return np.real(circuit(weights))

Loading the data which have been used to get the weights by previous training

In [9]:
value = np.loadtxt('8_qubit_data.out')
print(value)

[0.08284764 0.04822248 0.04772381 0.07697001 0.06667772 0.04918233
 0.07119205 0.06546178 0.0121052  0.03073409 0.0145168  0.0375386
 0.04186722 0.06653349 0.03505152 0.08634318 0.04634064 0.04056781
 0.06473209 0.06063315 0.06287014 0.07711482 0.03347822 0.02737753
 0.07375131 0.07332574 0.04090799 0.08015003 0.0373481  0.08558989
 0.0764542  0.0756789  0.08579295 0.07071218 0.06895074 0.07429418
 0.0817577  0.02078277 0.07063666 0.05202572 0.08695379 0.05972889
 0.04891439 0.07171632 0.07564259 0.0682474  0.06571066 0.06201586
 0.05695638 0.04054239 0.03041083 0.05193202 0.08022291 0.08732516
 0.05198229 0.0764754  0.07931452 0.08612146 0.05612533 0.08417969
 0.00314346 0.0846016  0.08704915 0.08413478 0.05536368 0.06300987
 0.03918602 0.07725638 0.02911662 0.05754946 0.02951604 0.04139544
 0.08602274 0.07291232 0.05108132 0.03970532 0.00536753 0.04533982
 0.0063251  0.08574067 0.06630697 0.08110905 0.07533784 0.05589753
 0.04224634 0.06460819 0.05390607 0.05812747 0.0684402  0.07721

Loading the weights to prepare the amplitude embedding of the above data

In [10]:
weights = np.loadtxt('8_qubit_weights.out')
print(weights)

[ 7.07129103e+00  1.25722207e+01  4.56983192e+00  2.36785476e+00
  1.57084698e+00  9.08351669e+00  1.13776435e+01  9.40771272e+00
  1.17221282e+01 -2.19482400e-03  4.74860056e+00  3.21313709e+00
  7.85353059e+00  9.42605276e+00  1.09674479e+01  9.42500297e+00
  9.48882159e+00  1.10216109e+01  4.59130100e+00  4.78753353e+00
  4.74872469e+00  1.10023146e+01  1.20897675e+00  1.09405914e+01
  6.31600285e+00  6.29365534e+00  3.14158555e+00  7.65431967e-01
 -5.62693376e-02  1.29409569e+01  9.44179920e+00 -5.29899179e-02]


We compite the fidelity of the generated states using VQSP and the original amplitude embedding needed.

The Cat qubit library does not support gathering state vector from circuit, so we get the probabilities of the result.

Since (Amplitude of state) ^ 2 = Probability,
we compute the squre root of Probability before finding the fidelity of the states.

In [12]:
predictions=variational_classifier(weights,value)
print(predictions)
# Amp (qml.state) ^ 2 = prob (qml.prob)
acc = accuracy(value, np.sqrt(predictions))
# print(predictions.numpy(), x.numpy())
print(acc)

[0.00390625 0.0078125  0.00097656 0.00097656 0.00390625 0.00292969
 0.00097656 0.00292969 0.00195312 0.00683594 0.00097656 0.00390625
 0.00488281 0.00097656 0.0078125  0.00097656 0.00292969 0.
 0.00585938 0.00390625 0.00683594 0.00097656 0.00390625 0.00683594
 0.00488281 0.00585938 0.00390625 0.00195312 0.00195312 0.00683594
 0.00097656 0.00292969 0.00585938 0.00292969 0.00097656 0.00195312
 0.00195312 0.0078125  0.00097656 0.00390625 0.00097656 0.00292969
 0.00292969 0.         0.00585938 0.00488281 0.00195312 0.00390625
 0.00585938 0.00683594 0.00292969 0.00585938 0.00488281 0.00683594
 0.00195312 0.00195312 0.00097656 0.00488281 0.00292969 0.00585938
 0.00585938 0.00683594 0.00292969 0.00878906 0.00390625 0.00488281
 0.00195312 0.00488281 0.00195312 0.         0.00097656 0.00390625
 0.00683594 0.00585938 0.00683594 0.00585938 0.00292969 0.
 0.00292969 0.00488281 0.00195312 0.00683594 0.00488281 0.00390625
 0.00292969 0.00195312 0.00683594 0.00195312 0.00585938 0.00292969
 0.00292969